### Задание 1.
Обязательная часть  
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time

In [21]:
KEYWORDS = ['python', 'парсинг']

In [22]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [23]:
posts = soup.find_all('article', class_='post post_preview')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/alexzfort/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/d2a/e8a/5ac/d2ae8a5ac9876f838ebdd45f3ea3bc3b.png" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">alexzfort</span>
 </a>
 <span class="post__time">сегодня в 00:35</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/537782/">Дайджест свежих материалов из мира фронтенда за последнюю неделю №450 (11 — 17 января 2021)</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/webdev/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Разработка веб-сайтов'); }" rel="nofo

In [24]:
posts_filtred = []
for post in posts:
    for key in KEYWORDS:
        if key in post.text.lower():
            posts_filtred.append(post)
            break

In [25]:
habr_request = pd.DataFrame()

for post in posts_filtred:
    date = post.find('span', class_='post__time').text 
    title = post.find('a', class_='post__title_link').text
    link = post.find('a', class_ = 'post__title_link').get('href')   

    
    row = {'date': date, 'title': title, 'link' : link}
    habr_request = pd.concat([habr_request, pd.DataFrame([row])])
#                 habr_news = pd.concat([habr_news, pd.DataFrame([row])], ignore_index=True)
habr_request.head()

,date,title,link
0,вчера в 22:37,Многопоточное скачивание файлов с ftp python-с...,https://habr.com/ru/post/537774/
0,вчера в 21:54,Нейросеть для раскрутки собачьего аккаунта в И...,https://habr.com/ru/post/537714/
0,вчера в 18:27,Прокачиваем скрипты симуляции HDL с помощью Py...,https://habr.com/ru/post/537704/


### Задание 2.

Обязательная часть  
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [2]:
EMAIL =  ['xxx@x.ru', 'yyy@y.com']

In [18]:
url = 'https://identityprotection.avast.com/v1/web/query/breaches-stats'

headers = {
# 'Accept': 'application/json, text/plain, */*',
# 'Accept-Encoding': 'gzip, deflate, br',
# 'Accept-Language': 'ru,en;q=0.9',
# 'Connection': 'keep-alive',
# 'Content-Length': '31',
# 'Host': 'identityprotection.avast.com',
# 'Origin': 'https://www.avast.com',
# 'Referer': 'https://www.avast.com/hackcheck/',
# 'Sec-Fetch-Dest': 'empty',
# 'Sec-Fetch-Mode': 'cors',
# 'Sec-Fetch-Site': 'same-site',
# 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.60 YaBrowser/20.12.0.966 Yowser/2.5 Safari/537.36',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Version': '0'
}

In [19]:
params = {
    'emailAddresses': EMAIL
}

In [42]:
res = requests.post(url,data=json.dumps(params),headers=headers)

In [60]:
res.json()

{'breaches': {'16613': {'breachId': 16613,
   'site': 'verifications.io',
   'recordsCount': 677914246,
   'description': 'Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked database contained information such as physical addresses, phone numbers, email addresses, dates of birth, gender, employer and job information, geographic location and IP addresses.\r\n\r\n',
   'publishDate': '2019-03-28T00:00:00Z',
   'statistics': {'usernames': 0, 'passwords': 0, 'emails': 677914246}},
  '17609': {'breachId': 17609,
   'site': 'vk.com',
   'recordsCount': 42536910,
   'description': 'At some time in 2020, the Russian social networking website vKontakte was allegedly breached. The stolen data contains email addresses and

In [69]:
df = pd.DataFrame(columns = ['почта', 'дата утечки', 'источник утечки', 'описание утечки'])

for site, email_breaches in res.json()['data'].items():
    for email, breaches in email_breaches.items():
        for breach in breaches:
            breach_info = res.json()['breaches'][str(breach['breachId'])]
            df.loc[len(df)] = [email, breach_info['publishDate'], breach_info['site'], breach_info['description']]         
df

,почта,дата утечки,источник утечки,описание утечки
0,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
1,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
2,yyy@y.com,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
3,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
4,yyy@y.com,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
5,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
6,yyy@y.com,2017-12-01T00:00:00Z,creocommunity.com,"At an unconfirmed date, Creo Community's user ..."
7,yyy@y.com,2019-07-11T00:00:00Z,medicaresupplement.com,"In May 2019, a security researcher discovered ..."
8,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
9,yyy@y.com,2020-11-19T00:00:00Z,123rf.com,"In March 2020, the stock image agency 123RF wa..."
